In [1]:
from my_trader import *
import pandas as pd
import numpy as np
from selenium import webdriver
import urllib2
import time
import re

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains


def create_driver_session(session_id, executor_url):
    from selenium.webdriver.remote.webdriver import WebDriver as RemoteWebDriver

    # Save the original function, so we can revert our patch
    org_command_execute = RemoteWebDriver.execute

    def new_command_execute(self, command, params=None):
        if command == "newSession":
            # Mock the response
            return {'success': 0, 'value': None, 'sessionId': session_id}
        else:
            return org_command_execute(self, command, params)

    # Patch the function before creating the driver object
    RemoteWebDriver.execute = new_command_execute

    new_driver = webdriver.Remote(command_executor=executor_url, desired_capabilities={})
    new_driver.session_id = session_id

    # Replace the patched function with original function
    RemoteWebDriver.execute = org_command_execute

    return new_driver

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/fix_yahoo_finance/__init__.py:43: DeprecationWarning: 
    Auto-overriding of pandas_datareader's get_data_yahoo() is deprecated and no longer available.
    Use pdr_override() to explicitly override it.
  DeprecationWarning)


In [3]:
driver=webdriver.Firefox()
session_id =driver.session_id
executor_url =driver.command_executor._url

In [163]:
driver.get('https://signon.onesource.com/Web/Logon/Signin.aspx')
time.sleep(5)
driver.implicitly_wait(5)
driver.find_element_by_id("username").clear()
driver.find_element_by_id("username").send_keys("hli@calasiancc.org")
driver.find_element_by_id("password").clear()
driver.find_element_by_id("password").send_keys("dbhoovers1")
driver.find_element_by_id("btnSignIn").click()
driver.implicitly_wait(5)
driver.find_element_by_link_text("Lists").click()

In [171]:
# list selector
i = 0
list_ele = driver.find_elements_by_class_name("name-row")
get = list_ele[0]
get.find_element_by_tag_name("a").click()

WebDriverException: Message: Element <a class="clickable" href="/company/c320c5f1-3ab7-3866-a7da-e86c7b2d5360"> is not clickable at point (223.65833282470703,414.66668701171875) because another element <span> obscures it


In [245]:
#page info

current_page = driver.find_element_by_class_name("current-page")
current_page=current_page.get_attribute('innerHTML')
current_page = int(current_page)

total_page = driver.find_element_by_class_name("total-page")
total_page=total_page.get_attribute('innerHTML')
total_page = int(total_page)


if total_page > current_page:
    next_page = driver.find_element_by_class_name("next-page")
    driver.execute_script("arguments[0].click();", next_page)
else:
    break

In [173]:
i = 1
company_list = driver.find_elements_by_class_name("clickable")
driver.implicitly_wait(5)
get_company = company_list[i]
company_list[i].click()

In [190]:
#get basic information
info = {}
for i in range(len(data)):
    data = driver.find_elements_by_class_name("data-container")
    try:
        label=data[i].find_element_by_class_name("data-label")
    except:
        continue 
        #label=data[i].find_element_by_class_name("data-label wide")
    value=data[i].find_elements_by_class_name("data-value")
    value = value[0].text
    label = label.text
    label = label.encode('ascii','ignore')
    info[label] = value
address = driver.find_element_by_class_name("company-address").text
info["address"] = address

In [193]:
data_title = ["description","industry","highlights"]
result=pd.DataFrame()
for i in data_title:
    get_temp = driver.find_element_by_class_name("highlights")
    get_temp = get_temp.text.split("\n")
    #get_temp = get_temp.text.split(":")
    for i in range(len(get_temp)):
        get_temp[i] = get_temp[i].encode('ascii','ignore')
        get = get_temp[i].split(":")
        result = result.append([get])
result = result.reindex()

In [246]:
contacts = driver.find_element_by_id("totalContacts").text
contacts = int(contacts)
if contacts != 0:
    driver.find_element_by_id("company_contacts").click()

WebDriverException: Message: Element <a id="company_contacts" class="report SearchReportType  company_contacts active" href="#report/company_contacts"> is not clickable at point (20.208335876464844,28.5) because another element <div class="name"> obscures it


In [268]:
# find contact
contacts = driver.find_element_by_id("totalContacts").text
contacts = int(contacts)
if contacts != 0:
    driver.find_element_by_id("company_contacts").click()
    contacts = driver.find_elements_by_class_name("detail-container")
#---------------------------
def render_text_contact(get_temp):
    #flag = False
    result = pd.DataFrame()
    get_temp = get_temp.text.split("\n")
    header=["name","position","location","industry"]
    count_header = 0
    #get_temp = get_temp.text.split(":")
    for i in range(len(get_temp)):
        get_temp[i] = get_temp[i].encode('ascii','ignore')
        if ":" in get_temp[i]:
            #flag = True
            get = get_temp[i].split(":")
            result = result.append([get])
            
        else:
            result = result.append([[header[count_header],get_temp[i]]])
            count_header +=1
        
    result = result.reindex()    
    return result
#---------------------------
contacts = driver.find_elements_by_class_name("search-result-container")
result = pd.DataFrame()
for i in range(len(contacts)):
    
    result = result.append(render_text(contacts[i]))

# reindexing

for i in range(len(result)):
    index_name = i/4
    result.iloc[i].name=index_name
x=range(len(result.index)/5 )
index_name=np.repeat(x,5)
result.index = index_name

# organize contact in to a dataframe
new_result = pd.DataFrame()
for i in list(set(result.index)):
    new_result= new_result.append([list(result.loc[i].T.iloc[1].values)])
new_result.columns=list(result.loc[0].T[0:1].iloc[0])  

In [494]:
new_result

,name,position,location,industry,Main
0,Miguel Gonzales,"President at Vallarta Supermarkets, Inc.","Pacoima, California, United States",Grocery Stores,818-485-0596
0,Rodrigo Gonzalez,"Vice President at Vallarta Supermarkets, Inc.","Pacoima, California, United States",Grocery Stores,818-485-0596
0,Adolfo Molina,"Vice President at Vallarta Supermarkets, Inc.","Pacoima, California, United States",Grocery Stores,818-485-0596
0,Jesus Torres,Director Information Technology at Vallarta Su...,"Pacoima, California, United States",Grocery Stores,818-485-0596
0,Moses Gonzalez,"Risk Manager at Vallarta Supermarkets, Inc.","Pacoima, California, United States",Grocery Stores,818-485-0596
0,John Sellers,"Buyer at Vallarta Supermarkets, Inc.","Pacoima, California, United States",Grocery Stores,818-485-0596


In [484]:
new_result

,name,position,location,industry,Main
0,Miguel Gonzales,"President at Vallarta Supermarkets, Inc.","Pacoima, California, United States",Grocery Stores,818-485-0596
0,Miguel Gonzales,"President at Vallarta Supermarkets, Inc.","Pacoima, California, United States",Grocery Stores,818-485-0596
0,Rodrigo Gonzalez,"Vice President at Vallarta Supermarkets, Inc.","Pacoima, California, United States",Grocery Stores,818-485-0596
0,Miguel Gonzales,"President at Vallarta Supermarkets, Inc.","Pacoima, California, United States",Grocery Stores,818-485-0596
0,Miguel Gonzales,"President at Vallarta Supermarkets, Inc.","Pacoima, California, United States",Grocery Stores,818-485-0596
0,Rodrigo Gonzalez,"Vice President at Vallarta Supermarkets, Inc.","Pacoima, California, United States",Grocery Stores,818-485-0596
0,Adolfo Molina,"Vice President at Vallarta Supermarkets, Inc.","Pacoima, California, United States",Grocery Stores,818-485-0596
0,Miguel Gonzales,"President at Vallarta Supermarkets, Inc.","Pacoima, California, United States",Grocery Stores,818-485-0596
0,Miguel Gonzales,"President at Vallarta Supermarkets, Inc.","Pacoima, California, United States",Grocery Stores,818-485-0596
0,Rodrigo Gonzalez,"Vice President at Vallarta Supermarkets, Inc.","Pacoima, California, United States",Grocery Stores,818-485-0596


In [476]:
new_result = pd.DataFrame()



,0,1,2,3,4
0,Miguel Gonzales,"President at Vallarta Supermarkets, Inc.","Pacoima, California, United States",Grocery Stores,818-485-0596


In [472]:
result.loc[i].T.iloc[1]

0                             Miguel Gonzales
0    President at Vallarta Supermarkets, Inc.
0          Pacoima, California, United States
0                              Grocery Stores
0                                818-485-0596
Name: 1, dtype: object